---

# À gauche, à droite, c'est un SOS!
### GoPiGo 101 - Série d'exercices 7
##### Manipulation du **servo-moteur**.

Un servo-moteur est un moteur asservi en position. Le moteur est équipé d'un capteur angulaire et d'un module d'asservissement lui permettant d'atteindre simplement une orientation donnée. Ainsi, il suffit de lui commander une position angulaire pour qu'il s'y déplace.

Pour cet exercice, vous devez avoir installé et connecté un servo-moteur sur l'un des ports servo (`SERVO1` ou `SERVO2`). [Voir cette image](https://gopigo3.readthedocs.io/en/master/api-basic/structure.html#hardware-ports).

---

Les fonctions suivantes permettent la manipulation des servo-moteurs :
 - `EasyGoPiGo3.init_servo` : crée un objet contrôleur du servo-moteur `easygopigo3.Servo`
 - `Servo.rotate_servo` : effectue la rotation du servo-moteur à l'angle donné
 - `Servo.reset_servo` : réinitialise le servo-moteur à sa position centrale
 - `Servo.disable_servo` : désactive l'asservissement du moteur et permet une manipulation manuelle

**Important** : Le robot GoPiGo3 peut contrôller deux servo-moteurs. Gardez en tête ces considérations :
 - la configuration mécanique actuelle du robot est :
     - `SERVO1`
         - se trouve sur la plateforme supérieure du robot
         - supporte la caméra
         - est connecté dans le port servo de droite
     - `SERVO2` 
         - se trouve sur la plateforme inférieure du robot
         - supporte le télémètre et le capteur lumineux
         - est connecté dans le port servo de gauche     
 - la rotation du servo est possible entre 0° et 180° :
     - 0° correspond à gauche complètement
     - 90° correspond au centre
     - 180° correspond à droite complètement
 - **mise en garde** : 
     - lorsque l'asservissement des servo-moteurs est effectif, il **ne faut pas** tenter de les bouger manuellement
     - il est possible de désactiver l'asservissement pour manuellement modifier la position des servo-moteurs avec la fonction `Servo.disable_servo`
 - **ATTENTION - IMPORTANT** : **vous devez obligatoirement limiter la rotation des servo-moteurs par programmation en tout temps** :
     - 45° à 135° pour la caméra sur le port `SERVO1`
         - 90° ± 45°
     - 35° à 145° pour le télémètre sur le port `SERVO2`
         - 90° ± 55°
 - la configuration mécanique des robots fait en sorte que l'opération des servo-moteurs est délicate
     - le câble de la caméra est tendu lorsque le servo-moteur est trop tourné
     - le côté du télémètre ente en collision avec le chassis du robot si le servo-moteur est trop tourné
 - Finalement, on remarque que l'ajustement mécanique du zéro n'est pas parfait. Il faudra trouver une façon de faire ce 0 par programmation.

### Démonstration

In [5]:
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()

camera_servo_port = 'SERVO1'
range_sensor_servo_port = 'SERVO2'

camera_servo_control = robot.init_servo(port=camera_servo_port)
range_sensor_servo_control = robot.init_servo(port=range_sensor_servo_port)

time_to_sleep = 1.

# Recentre les 2 servos
camera_servo_control.reset_servo()
range_sensor_servo_control.reset_servo()
time.sleep(time_to_sleep)

# Applique une rotation opposée sur les 2 servos
camera_servo_control.rotate_servo(135)
range_sensor_servo_control.rotate_servo(35)
time.sleep(time_to_sleep)


# dégagement de l'asservissement du servo de la caméra pendant un certain temps 
# permet un déplacement manuel
time_to_displacement = 12
camera_servo_control.disable_servo()
for i in reversed(range(time_to_displacement)):
    print(f'\rVous avez {i} seconde{"s" if i > 1 else ""} pour déplacer manuellement le servo-moteur de la CAMÉRA... avant son recentrage.', end='')
    time.sleep(1.)
camera_servo_control.reset_servo()

# dégagement de l'asservissement du servo du télémètre pendant un certain temps 
# permet un déplacement manuel

range_sensor_servo_control.disable_servo()
for i in reversed(range(time_to_displacement)):
    print(f'\rVous avez {i} seconde{"s" if i > 1 else ""} pour déplacer manuellement le servo-moteur du TÉLÉMÈTRE... avant son recentrage.', end='')
    time.sleep(1.)
range_sensor_servo_control.reset_servo()

range_sensor_servo_control.rotate_servo(5)

del range_sensor_servo_control
del camera_servo_control
del range_sensor_servo_port
del camera_servo_port
del robot
del time
del gpg

Vous avez 0 seconde pour déplacer manuellement le servo-moteur du TÉLÉMÈTRE... avant son recentrage...

---
### Préparation

In [22]:
import easygopigo3 as gpg
robot = gpg.EasyGoPiGo3()
import time
camera_servo_port = 'SERVO1'
range_sensor_servo_port = 'SERVO2'

camera_offset_correction = 1 
range_sensor_offset_correction = -10

camera_servo_control = robot.init_servo(port=camera_servo_port)
range_sensor_servo_control = robot.init_servo(port=range_sensor_servo_port)

def reset_servos():
    range_sensor_servo_control.rotate_servo(80)
    camera_servo_control.rotate_servo(91)

def perf_sleep(for_n_seconds : int = 0) -> None:
    time_start = time.perf_counter()
    time_counter = 0
    time_to_wait = for_n_seconds
    while (time_counter < time_to_wait):
        time_counter = time.perf_counter() - time_start
        

reset_servos()

---
### Exercice 7.1.
Faites un parcours de 45° à 135° par incrément de 5° sur le servo de la caméra alors que le servo du télémètre effectue le parcours inverse (de 135° à 45°). Vous devez attendre 0.25 sec entre chaque pas du parcours.

In [3]:
start_time = time.perf_counter()
elapsed_time = 0
n_cycles= 18

camera_offset_correction = 1 
range_sensor_offset_correction = -10
camera_servo_control.rotate_servo(45 + camera_offset_correction)
range_sensor_servo_control.rotate_servo(135 + range_sensor_offset_correction)

for i in range(n_cycles):
    camera_servo_control.rotate_servo(45 + camera_offset_correction + i * 5 )
    range_sensor_servo_control.rotate_servo(135 + range_sensor_offset_correction - i * 5)
    perf_sleep(0.25)
    
reset_servos()

---
### Exercice 7.2.
Faites le même parcours qu'au numéro précédent tout en enchênant le parcours inverse. C'est-à-dire faire de 45° à 135° puis à 45° puis à 135° et ainsi de suite par incrément de 5°.

Vous devez faire une fonction qui fait n cycles avec cette signature : 
<br>`def move_servos(n_cycle=2, time_to_wait=0.25)`

Assurez-vous que les 2 sevos soient en position opposée.

In [9]:
cycles = int((135 - 45) / 5)

def move_servos(n_cycle=2, time_to_wait=0.25):
    for _ in range(n_cycle):
        for i in range(cycles):
            camera_servo_control.rotate_servo(45 + camera_offset_correction + i * 5 )
            range_sensor_servo_control.rotate_servo(135 + range_sensor_offset_correction - i * 5)
            perf_sleep(0.25)
        for i in range(cycles):
            camera_servo_control.rotate_servo(135 + camera_offset_correction - i * 5 )
            range_sensor_servo_control.rotate_servo(45 + range_sensor_offset_correction + i * 5)
            perf_sleep(0.25)
            
move_servos()
reset_servos()
    

NameError: name 'reset_servos' is not defined

---
### Exercice 7.3.
Comme vous l'avez sans doute remarqué, l'angle 90° qui devrait être parfaitement aligné avec le devant du robot ne l'est pas tout à fait. Une limitation mécanique limite l'ajustement physique possible. Il faut donc remédier à cette solution par logiciel.

Faites un petit programme qui utilise la télécommande pour trouver l'angle qui est le mieux aligné avec le devant du robot. Faites afficher l'angle courant afin de déterminer facilement l'angle obtenu lors du zéro mécanique.

Limitez la recherche à 90° ± 45°. 

Prenez en note les informations obtenues, elles seront réutiliées.

In [11]:
reset_servos()

---
### Exercice 7.4.
Suivant les informations obtenues au numéro précédent, on vous demande de faire la fonction suivante :
 - faire une fonction qui déplace un servo selon cette signature :
 - `def servo_position(servo_id, angle)`
 - l'angle est l'orientation désirée, par contre, on veut changer de référentiel, au lieu d'avoir les angles variants de 0° à 180°, on désire que l'angle soit défini ainsi :
     - 0° droit devant le robot
     - 90° à droite
     - -90° à gauche
     - toutefois, l'angle doit être limité à ±45°
     - finalement, vous devez utiliser les zéros obtenu au numéro précédent
 - le `servo_id` est l'identifiant :
     - `0` la caméra
     - `1` le télémètre
     - on utilise la caméra pour toutes autres valeurs

<pre>

                            90°                                                                0°  * le zéro mécanique imparfait                       
                             ^                                                                 ^     est corrigé par logiciel                   
                             |                            ____|\                               |                        
                             |                           !      \                              |   * un changement de référentiel                     
                             |                           !       \                             |     est appliqué
                             |                           !       /                             |                        
                             |                           |____  /                              |                        
                             |                                |/                               |                        
                             |                                                                 |                        
                             |                                                                 |                        
   180° <--------------------+--------------------> 0°               -90° <--------------------+--------------------> 90°
</pre>

**Attention** : la réalisation de cet exercice peut occasionné des dépassements de la plage limite. Il est donc conseillé de faire imprimé le résultat à l'écran de vos angles avant de les utiliser directement sur le robot et de faire des erreurs de dépassement.

Lorsque vous fonction est terminée, déplacer les servos comme au numéro 1. Assurez-vous de centrer les servos à la fin.

In [21]:
def servo_positions(servo_id:int, angle:int):
    if not -45 < angle < 45:
        raise ValueError("Vous dépassez les bornes")
    else:
        angle += 90
        if servo_id == 0:
            camera_servo_control.rotate_servo(angle + camera_offset_correction)
        elif servo_id == 1
            range_sensor_servo_control.rotate_servo(angle + range_sensor_offset_correction)  
        else:
            raise ValueError("You suck, your ID is so bad")

servo_positions(0, 0)

ValueError: Vous dépassez les bornes

---
### Exercice 7.5. * §
Faites une classe qui encapsule l'opération d'un servo moteur. Cette classe doit :
 - prendre dans son constructeur les paramètres suivants :
     - le robot
     - si on s'adresse au servo de la caméra ou à celui du télémètre
     - le zéro
     - les deux limites d'opération : minimum et maximum (ATTENTION, ces limites doivent s'auto-limiter à un maximum de ±55°)
 - il doit être possible de demander le déplacement de la position désirée - toutefois, la position est toujours référencée selon le zéro et ne dépasse jamais les limites imposées
 - en tout temps, il doit être possible de questionner cette classe pour connaître la position du servo
 - n'oubliez pas qu'on désire toujours le 0° devant le robot avec un ± _x_° pour aller de droite(+) à gauche(-)

In [ ]:
# Solution
# ...